## Reduction operation: the sum of the numbers in the range [0, value)

In [1]:
import numpy as np

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial

value = 5*10**4

X = np.random.rand(value)

# Para imprimir los pimeros valores del array

# print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)


print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)

print("Now, the result using numpy.ndarray.sum():", X.sum())




Time taken by reduction operation using a function: 44.7 ms ± 39.9 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
And the result of the sum of numbers in the range [0, value) is: 24991.51788404963

Time taken by reduction operation using numpy.sum(): 96.3 µs ± 1.54 µs per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.sum(): 24991.51788404961 
 
Time taken by reduction operation using numpy.ndarray.sum(): 72.3 µs ± 367 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.ndarray.sum(): 24991.51788404961


### 3.2 (A)

In [9]:
import numpy as np
from multiprocessing import Pool
import time

def reduc_operation(A):
    """Compute the sum of the elements of Array A."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Generar el array original
value = 5 * 10**6
X = np.random.rand(value)

# Función para dividir el array en partes
def split_array(array, num_splits):
    """Divide el array en `num_splits` partes iguales."""
    split_size = len(array) // num_splits
    return [array[i * split_size: (i + 1) * split_size] for i in range(num_splits)]

# Tiempo para la reducción con un solo proceso
start_time = time.time()
result_single = reduc_operation(X)
end_time = time.time()
print(f"Tiempo con 1 proceso: {end_time - start_time:.6f} segundos, Suma: {result_single}")

# Tiempo para la reducción con 2 procesos
splits_2 = split_array(X, 2)
start_time = time.time()
with Pool(2) as pool:
    results_2 = pool.map(reduc_operation, splits_2)
result_two_processes = sum(results_2)
end_time = time.time()
print(f"Tiempo con 2 procesos: {end_time - start_time:.6f} segundos, Suma: {result_two_processes}")

# Tiempo para la reducción con 4 procesos
splits_4 = split_array(X, 4)
start_time = time.time()
with Pool(4) as pool:
    results_4 = pool.map(reduc_operation, splits_4)
result_four_processes = sum(results_4)
end_time = time.time()
print(f"Tiempo con 4 procesos: {end_time - start_time:.6f} segundos, Suma: {result_four_processes}")

Tiempo con 1 proceso: 5.034564 segundos, Suma: 2499993.6375147984
Tiempo con 2 procesos: 2.973715 segundos, Suma: 2499993.637514972
Tiempo con 4 procesos: 1.649338 segundos, Suma: 2499993.637514888


- Nota: Cuando el value es el que viene por defecto (5 * 10**4) apenas hay diferencias significativas en el tiempo
  cuando se utilizan más CPUs. Sin embargo, conforme se aumenta el valor de este de manera exponencial (10**5, 10**6, etx) la diferencia se hace mucho    más significatia y apreciable. 

### 3.2 (B)

In [13]:
import numpy as np
from numba import njit, prange
import time

# Función original sin optimización
def reduc_operation(A):
    """Compute the sum of the elements of Array A."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Versión optimizada con Numba (sin paralelismo)
@njit
def reduc_operation_numba(A):
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Versión optimizada con Numba y paralelismo
@njit(parallel=True)
def reduc_operation_numba_parallel(A):
    s = 0
    for i in prange(A.size):
        s += A[i]
    return s

# Tamaño del array
value = 5*10**7
X = np.random.rand(value)

# Medir tiempos
def measure_time(func, array, label):
    start_time = time.time()
    result = func(array)
    end_time = time.time()
    print(f"Tiempo con {label}: {end_time - start_time:.6f} segundos, Suma: {result}")

print(f"Array size: {value}\n")

# Tiempo con función original
measure_time(reduc_operation, X, "función original")

# Tiempo con Numba (sin paralelismo)
measure_time(reduc_operation_numba, X, "Numba sin paralelismo")

# Tiempo con Numba (paralelismo)
measure_time(reduc_operation_numba_parallel, X, "Numba con paralelismo")

Array size: 50000000

Tiempo con función original: 52.847927 segundos, Suma: 25001489.472300984
Tiempo con Numba sin paralelismo: 0.709224 segundos, Suma: 25001489.472300984
Tiempo con Numba con paralelismo: 1.892220 segundos, Suma: 25001489.47229185


- Nota: Como ocurría en el apartado anterior, cuando el value es el que viene por defecto (5 * 10**4) apenas hay diferencias significativas en el tiempo
 de ejecución. Sin embargo, conforme se aumenta el valor de este de manera exponencial (10**5, 10**6, etx) la diferencia se hace mucho más significatia y apreciable. Disminuyendo de 52 segungos con la opción por defecto a 0.7 segundos usando Numba. El uso de paralelismo también disminuye muy significativament el tiempo, aunque no tanto como el no paralelismo. Posiblemente se deba a que, puesto que el script es relativamente sencillo, se tarda más en cargar los módulos y preparar el paralelismo que en su ejecución en sí misma. Si el proceso fuese más tedioso computacionalmente la diferencia sí se acentuaría más. 